<a id='top'></a>

# CSCI 3022: Intro to Data Science - Spring 2020 Practicum 2
***

This practicum is due on Canvas by **11:59 PM on Friday May 1**. Your solutions to theoretical questions should be done in Markdown/MathJax directly below the associated question.  Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  

**Here are the rules:** 

1. All work, code and analysis, must be your own. 
1. You may use your course notes, posted lecture slides, textbooks, in-class notebooks, and homework solutions as resources.  You may also search online for answers to general knowledge questions like the form of a probability distribution function or how to perform a particular operation in Python/Pandas. 
1. This is meant to be like a coding portion of your midterm exam. So, the instructional team will be much less helpful than we typically are with homework. For example, we will not check answers, help debug your code, and so on.
1. If something is left open-ended, it is because we want to see how you approach the kinds of problems you will encounter in the wild, where it will not always be clear what sort of tests/methods should be applied. Feel free to ask clarifying questions though.
2. You may **NOT** post to message boards or other online resources asking for help.
3. You may **NOT** copy-paste solutions *from anywhere*.
4. You may **NOT** collaborate with classmates or anyone else.
5. In short, **your work must be your own**. It really is that simple.

Violation of the above rules will result in an immediate academic sanction (*at the very least*, you will receive a 0 on this practicum or an F in the course, depending on severity), and a trip to the Honor Code Council.

**By submitting this assignment, you agree to abide by the rules given above.**

***

**Name**:  Alex Mazur

***


**NOTES**: 

- You may not use late days on the practicums nor can you drop your practicum grades. 
- If you have a question for us, post it as a **PRIVATE** message on Piazza.  If we decide that the question is appropriate for the entire class, then we will add it to a Practicum clarifications thread. 
- Do **NOT** load or use any Python packages that are not available in Anaconda 3.6. 
- Some problems with code may be autograded.  If we provide a function API **do not** change it.  If we do not provide a function API then you're free to structure your code however you like. 
- Submit only this Jupyter notebook to Canvas.  Do not compress it using tar, rar, zip, etc. 
- This should go without saying, but... For any question that asks you to calculate something, you **must show all work to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit.


---

In [1]:
from scipy import stats
from math import isnan
import numpy as np 
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from calendar import month_name, different_locale

<br>

---

### [50 points] Problem 1: Multiple Linear Regression to Explain House Hauntings

<img src="https://s-media-cache-ak0.pinimg.com/originals/09/72/01/09720128cff5de4d4af038cd3fcf7f69.jpg" style="width: 300px;"/>

In an effort to control the skyrocketing prices of real estate in the Colorado Front Range, Governor Polis implemented a cutting edge new intervention. This new program oversaw the introduction of ghosts back into their natural ecosystem, after the ghost population seriously dwindled in recent decades due to overhaunting. However, an unfortunate miscalculation has led to haunted houses becoming a very serious problem in Colorado. Modern problems require modern solutions, so Governor Polis has hired you and the famous hedgehog data scientist/part-time ghostbuster Amy to determine what features of a house may be used to best predict a `haunted` score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted).

You decide to use multiple linear regression to understand and predict what factors lead to increased haunted house hazard. You collected a data set from Haunted Zillow, the lesser-known database of haunted house prices and attributes. The data cover a variety of potential features, and you'll find this data in the file `houses.csv`. 

**Response**: 

- $\texttt{haunted}$: a haunting score, related to the probability that a house with the given features is haunted (higher $\leftrightarrow$ more likely to be haunted)

**Features**: 

- $\texttt{age}$: age of the house, in years
- $\texttt{area}$: square footage of interior of house
- $\texttt{bathrooms}$: number of bathrooms
- $\texttt{distance metro}$: distance to the nearest major metropolitan area (in miles)
- $\texttt{distance cemetery}$: distance to the nearest cemetery (in miles)
- $\texttt{cats}$: the number of cats within a one-block radius of the house
- $\texttt{howls}$: the number of wolf howls heard on an average night in the house's neighborhood
- $\texttt{clouds}$: what percentage of the sky was covered by clouds (fraction, 0-1)
- $\texttt{precipitation}$: amount of precipitation in the past 72 hours (inches)
- $\texttt{misery index}$: an economic indicator for how miserable the average United States citizen is, based on the unemployment rate and the inflation rate. More [here](https://www.stuffyoushouldknow.com/podcasts/whats-the-misery-index.htm) and [here](https://en.wikipedia.org/wiki/Misery_index_(economics)). Higher values correspond to more miserable citizens.
- $\texttt{ice cream sold}$: the number of units of ice cream sold at the farmer's market the week the house was most recently sold

**Part A**: Read the data from `houses.csv` into a Pandas DataFrame.  Note that since we will be doing a multiple linear regression we will need all of the features, so you should drop any row in the DataFrame that is missing data. 

In [265]:
# Your code here.
houses = pd.read_csv('houses.csv')
houses.dropna(inplace=True)
houses.head()

,age,area,bathrooms,distance metro,distance cemetery,cats,howls,clouds,precipitation,misery index,ice cream sold,haunted
0,118.94,2113.0,4,7.1,3.23,7.0,3.0,1.00,0.82,12.99,273.0,0.123131
1,20.60,1773.0,2,7.4,4.19,5.0,5.0,1.00,0.99,16.77,184.0,-0.200243
2,260.84,2511.0,3,7.0,3.38,2.0,0.0,1.00,1.17,16.49,141.0,0.258651
3,101.33,2000.0,2,7.9,4.05,6.0,8.0,0.13,0.92,8.28,146.0,0.147983
4,96.94,1476.0,1,7.5,5.75,4.0,1.0,1.00,1.73,5.90,178.0,-0.113756


**Part B**: Perform the appropriate statistical test at the $\alpha = 0.01$ significance level to determine if _at least one_ of the features is related to the the response $y$.  Clearly describe your methodology and show all computations in Python. 

step 1: run ols model, haunted as response, everything else as the predictors. 

step 2: look at results for the p-value of the f-stat

step 3: we reject the null if the p-value for the f-stat is lower than 0.01 and at least one of the values is sig

In [266]:
feature = houses[houses.columns[0:11]]
response = houses['haunted']

feature = sm.add_constant(feature)
model = sm.OLS(response, feature)
result = model.fit()
print("The p-value for the f-statistic is: " + str(result.f_pvalue))

The p-value for the f-statistic is: 3.754240101428311e-14


**The F-Stat is less than 0.01, so we can reject our null hypothesis and say that at least one value is significant.**

**Part C**: Write a function `forward_select(df, resp_str, maxk)` that takes in the DataFrame, the name of the column corresponding to the response, and the maximum number of desired features, and returns a list of feature names corresponding to the `maxk` most important features via forward selection.  At each stage in forward selection you should add the feature whose inclusion in the model would result in the lowest sum of squared errors $(SSE)$. Use your function to determine the best $k=5$ features to include in the model. Clearly indicate which feature was added in each stage. 

**Note**: The point of this exercise is to see if you can implement **foward_select** yourself.  You may of course use canned routines like statmodels OLS, but you may not call any Python method that explicitly performs forward selection.

step 1: Write an algo! (distance cemetery should be the first one)

step 2: for all the predictors run OLS on each column against haunted

step 3: each one will have a Mean squared error, we want to keep the lowest one

step 4: run linear regression with 2 predictors this time, whatever had the lowest MSE from before and all the other columns

step 5: keep the lowest MSE from above and run that again until we have 5 predictors

In [267]:
# Your code here.
def forward_select(df, resp_str, maxk):
    response = df[resp_str]
    lowestMSE = float('inf')
    lowestColumn = ''
    
    #first find the lowest MSE column
    for column in df:
        if column != resp_str:
            feature = df[column]
            feature = sm.add_constant(feature)
            result = sm.OLS(response, feature).fit()
            if lowestMSE > result.mse_resid:
                lowestMSE = result.mse_resid
                lowestColumn = column
    modelList = [lowestColumn]
    
    #Then find the rest of the columns
    for i in range(1,maxk):
        lowestMSE = float('inf')
        for column in df:
            if column in modelList:
                continue
            tempList = []
            tempList = modelList
            tempList.append(column)
            if column != resp_str:
                feature = df[tempList]
                feature = sm.add_constant(feature)
                result = sm.OLS(response, feature).fit()
                if lowestMSE > result.mse_resid:
                    lowestMSE = result.mse_resid
                    lowestColumn = column
            tempList.pop()
        modelList.append(lowestColumn)
    #Return a model!
    return modelList
            
print("The results of calling forward_select() on the houses dataframe are as follows: \n")
print(forward_select(houses,'haunted',5))

The results of calling forward_select() on the houses dataframe are as follows: 

['distance cemetery', 'cats', 'age', 'misery index', 'distance metro']


**Part D**: Write down the multiple linear regression model, including estimated parameters, obtained by your forward selection process. 

I got haunted ~ distance cemetery + cats + age + misery index + distance metro 

**Part E**: Perform the appropriate statistical test at the $\alpha = 0.05$ significance level to determine whether there is a statistically significant difference between the full model with all features and the reduced model obtained by forward selection in **Part D**. 

step 1: use full OLS model and reduced OLS model and run them in https://www.statsmodels.org/stable/generated/statsmodels.stats.anova.anova_lm.html

step 2: pr(>f), a value larger than 0.05 fails to reject the null, which is reduced model is suffecient.

In [268]:
# Your code here.
from statsmodels.formula.api import ols

houses.columns = ["age", "area", "bathrooms", "distance_metro", "distance_cemetery", "cats", "howls", "clouds", "precipitation", "misery_index", "ice_cream_sold", "haunted"]

fullmodel = ols(formula='haunted ~ age + area + bathrooms + distance_metro + distance_cemetery + cats + clouds + precipitation + misery_index + ice_cream_sold', data=houses).fit()
reduModel = ols(formula='haunted ~ age + distance_metro + distance_cemetery + cats + misery_index', data=houses).fit()

results = sm.stats.anova_lm(reduModel, fullModel)
print(results)
houses.columns = ["age", "area", "bathrooms", "distance metro", "distance cemetery", "cats", "howls", "clouds", "precipitation", "misery index", "ice cream sold", "haunted"]

   df_resid       ssr  df_diff   ss_diff         F   Pr(>F)
0      66.0  3.826630      0.0       NaN       NaN      NaN
1      60.0  3.743052      6.0  0.083578  0.223289  0.96774


C:\Users\alexm\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\alexm\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\alexm\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


**Pr(>F) is greater than 0.05. We will fail to reject the null and use the reduced model going forward.**

**Part F**: Based on your conclusions in **Part E**, use the _better_ of the two models to predict the haunted house hazard when the following features are observed: 

- $\texttt{age}$: 150 years
- $\texttt{area}$: 2200 square feet
- $\texttt{bathrooms}$: 3 bathrooms
- $\texttt{distance metro}$: 25 miles
- $\texttt{distance cemetery}$: 5 miles
- $\texttt{cats}$: 20 cats
- $\texttt{howls}$: 5 wolf howls/night
- $\texttt{clouds}$: 0.65 cloud cover
- $\texttt{precipitation}$: 0 inches
- $\texttt{misery index}$: 10
- $\texttt{ice cream sold}$: 125

step 1: Make a model with the reduced column set from above

step 2: Put constants into a prediction formula and determine haunted's value

In [269]:
# Your code here.

#Build reduced OLS model
feature = houses[forward_select(houses,'haunted',5)]
response = houses['haunted']

feature = sm.add_constant(feature)
model = sm.OLS(response, feature)
reducedResults = model.fit()
prediction = reducedResults.params[0] + (reducedResults.params[1]*5) + (reducedResults.params[2]*20) + (reducedResults.params[3]*150) + (reducedResults.params[4]*10) + (reducedResults.params[5]*25)

print("Haunted prediction given above values: " + str(prediction))

Haunted prediction given above values: 0.09486332071744463


**Part G:** Governor Polis dabbles a bit in the art of data science, as well as the science of data art. He tells you that the response (`haunted` score) that you and Amy predicted is actually the natural logarithm of the _odds_ that a house with the given features is haunted, where if $p$ is the probability that a house is haunted, then the odds are given by $$\text{odds} = \dfrac{p}{1-p}$$

So using Govenor Polis's information and combining it with our knowledge of logistic regression, the probability of a house being haunted is:

$$p(\text{haunted} \mid x_1, x_2, \ldots) = \dfrac{1}{1+\texttt{exp}[-(\beta_0 + \beta_1x_1 + \ldots + \beta_p x_p)]}$$

Perform this computation, then use a decision threshold of 0.5 to classify the house from **Part F** as haunted or not haunted. No new models should be fit here; use the same model that you used in Part F.

In [270]:
# Your code here.
import math
prob = 1/(1 + math.exp(-prediction))
print(prob)

0.5236980611752294


Since the value is > 0.5 we can clasify the house from Part F as haunted!

<br>

---



### [50 points] Problem 2: Amazon Forest Fires
A non-profit trying to protect the amazon rain forest has recruited you to join their data science corps. For your first task, they've given you a dataset with the number of reported forest fires in each state in the Amazon region of Brazil during each month between 1998 and 2017. The Brazilian government has 500 extra wildland firefighters and they have asked your non-profit to determine which state or states they should allocate these firefighters to during each month of the year. To do this, they want you to calculate an 80% confidence interval for the mean and median number of fires that occur during each month for each state, and use those statistics to determine where the firefighters should be assigned.

#### Part A: Loading The CSV
Read the csv located in `amazon.csv` into a pandas data frame. Brazil and many other countries use the period (.) symbol as a thousands separator and a comma (,) as the decimal separator. Ex. One Thousand And $\frac{75}{100}$ would be represented as $1.000,75$ instead of the familiar english notation $1,000.75$. When you read it in, you'll need to use a period(.) as the thousands separator and a comma(,) as the decimal separator. Because the comma is already in use as the decimal separator, this file uses a different character to separate columns in the data. Open up the file in a text editor and figure out what character was used. Then find the correct arguments to `pd.read_csv` to read in this file properly. Look up the docs if you're unsure what the arguments you'll need are. Print out the `.info` summary of the dataframe after you've read it in.

In [271]:
amazon_fires = pd.read_csv('amazon.csv', sep = ';', thousands = '.', decimal = ',')
amazon_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    6453 non-null   object 
 1   state   6452 non-null   object 
 2   month   6454 non-null   object 
 3   number  6448 non-null   float64
 4   date    6450 non-null   object 
dtypes: float64(1), object(4)
memory usage: 252.2+ KB


#### Part B: Data Cleaning

This dataset isn't paticularly useful in it's current state, so we'll need to clean it up a bit. Some data scientists say that most of their job is to wrangle data, so this will give you a taste of cleaning a real world data set. Perform the following tasks. 
1. Drop the 'date' column. The only information this column holds is the year, which we already have in another column. Use the `.info` summary provided to check your work.
2. Drop any rows with null values in any of the remaining columns. Use the provided code to print the number of rows remaining after this step.
3. Print all the unique values of the 'month' column. You'll notice that one is encoded with a different character encoding then the format that pandas is using.
4. Convert the Portugese month names to English month names. If you'd like to use them, we've included the 'month_name' and the 'different_encoding' modules of the python calendar library. There are many ways to accomplish this task, and these modules are not required, but may make things easier. As part of this step, you should make sure that the Portugese month with the encoding problem is translated to the correct english month. Use the `.unique` method provided for you to check your work. 
5. Check the number column for any values that seem impossible. If you find any values you think are impossible, drop them. As a guidline, we would never expect a single state to have more than 50,000 reported forest fires in a single month. Also keep in mind that we are tracking forest fires here. Do negative or fractional forest fires really make sense? You should check for any obivously impossible conditions that you think might occur, and drop rows accordingly. Use the provided code to print the number of rows remaining after this step.
6. Since you're new on the job, some of your co-workers may have played a prank on you... Print out all the unique values of the 'year' column and drop any rows with values that don't make sense. Use the provided code to print the number of rows remaining after this step.
7. For every state in the data, print the number of rows the state has associated with it. A number of states have far more observations than the others. Each state should have roughly 240 observations (20 years multiplied by 12 months/year minus any bad data). Drop all the observations for any states that have more than 240 rows associated with them. For two points of extra credit, figure out why these states have way more rows associated with them than they should. If you choose to do the extra credit, put your answer in the markdown cell below. 
8. To give you an idea of whether your answer is correct, we've provided a unit test below the last cell. It should pass. If it doesn't, go back and figure out which step has gone awry.

We've given you a code cell for each task to make organizing the grading a bit easier. Please perform step 1 in the first code cell and so on.

**NOTE:** Since some of these tasks are not totally trivial, you may use any resources other than your classmates on this part of this problem. This means you may consult google, stack overflow, the python/pandas documentation, some random book on pandas you might have, etc... But you may not consult your classmates for help. We will also be more helpful on this problem in office hours and in response to your *private* piazza messages.  ***CITE ALL RESOURCES USED IN A CODE COMMENT. A URL OR A BOOK TITLE IS SUFFICIENT. ANY CODE OBIVOUSLY COPIED FROM OUTSIDE SOURCES WITH OUT A CITATION WILL EARN YOU NO CREDIT ON THIS PROBLEM.***

In [272]:
#Code for data cleaning task 1 here:
amazon_fires.drop(['date'], axis=1, inplace=True)
amazon_fires.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6454 entries, 0 to 6453
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    6453 non-null   object 
 1   state   6452 non-null   object 
 2   month   6454 non-null   object 
 3   number  6448 non-null   float64
dtypes: float64(1), object(3)
memory usage: 201.8+ KB


In [273]:
#Code for data cleaning task 2 here:
print("Pre dropna: " + str(len(amazon_fires)))
amazon_fires.dropna(inplace=True)
print("Post dropna: " + str(len(amazon_fires)))

Pre dropna: 6454
Post dropna: 6446


In [274]:
#Code for data cleaning task 3 here:
print(amazon_fires['month'].unique())

['Janeiro' 'Fevereiro' 'Mar�o' 'Abril' 'Maio' 'Junho' 'Julho' 'Agosto'
 'Setembro' 'Outubro' 'Novembro' 'Dezembro']


In [275]:
#Code for data cleaning task 4 here:
def translate_month(month):
    if (month == "Janeiro"):
        month = "January"
    elif (month == "Fevereiro"):
        month = "February"
    elif (month == "Abril"):
        month = "April"
    elif (month == "Maio"):
        month = "May"
    elif (month == "Junho"):
        month = "June"
    elif (month == "Julho"):
        month = "July"
    elif (month == "Agosto"):
        month = "August"
    elif (month == "Setembro"):
        month = "September"
    elif (month == "Outubro"):
        month = "October"
    elif (month == "Novembro"):
        month = "November"
    elif (month == "Dezembro"):
        month = "December"
    else:
        month = "March"
    return month

amazon_fires['month'] = amazon_fires['month'].apply(translate_month)
print(amazon_fires['month'].unique())

['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'October' 'November' 'December']


In [276]:
#Code for data cleaning task 5
#Find the rows that are an integer between 0 and 50,000 and then take the rows we didn't find and remove those.
toDrop = amazon_fires[(~amazon_fires['number'].isin(range(0,50001)))].index
amazon_fires = amazon_fires.drop(toDrop)

print(len(amazon_fires))

6441


In [277]:
#Code for data cleaning task 6
def verifyYear(year):
    year = str(year)
    if year.isdigit() == False:
        year = -1
    return int(year)
print(amazon_fires['year'].unique())

amazon_fires['year'] = amazon_fires['year'].apply(verifyYear)

toDrop = amazon_fires[(amazon_fires['year'] == -1)].index
amazon_fires = amazon_fires.drop(toDrop)

print(amazon_fires['year'].unique())

print(len(amazon_fires))

['1998' '1999' '2000' '2001' '2002' '2003' '2004' '2005' '2006' '2007'
 '2008' '2009' '2010' '2011' '2012' '2013' '2014' '2015' '2016' '2017'
 '1000bc' '-40' '10bc' "our new data scientist won't notice this"]
[1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011
 2012 2013 2014 2015 2016 2017]
6437


In [279]:
#Code for data cleaning task 7
uniqueStates = amazon_fires['state'].unique()
for state in uniqueStates:
    stateQ = amazon_fires[(amazon_fires['state'] == state)].index
    numberOfObs = len(stateQ)
    print(state + " has: " + str(numberOfObs) + " entries.")
    if numberOfObs > 240:
        amazon_fires = amazon_fires.drop(stateQ)
print()
print("After dropping states with over 240 entries!")
print()

for state in uniqueStates:
    stateQ = amazon_fires[(amazon_fires['state'] == state)].index
    numberOfObs = len(stateQ)
    print(state + " has: " + str(numberOfObs) + " entries.")
    if numberOfObs > 240:
        stateQ = stateQ[(numberOfObs - 240):]
        amazon_fires = amazon_fires.drop(stateQ)
        
print(len(amazon_fires))

Acre has: 239 entries.
Alagoas has: 240 entries.
Amapa has: 239 entries.
Amazonas has: 239 entries.
Bahia has: 239 entries.
Ceara has: 239 entries.
Distrito Federal has: 239 entries.
Espirito Santo has: 238 entries.
Goias has: 238 entries.
Maranhao has: 239 entries.
Mato Grosso has: 476 entries.
Minas Gerais has: 237 entries.
Para has: 235 entries.
Paraiba has: 474 entries.
Pernambuco has: 239 entries.
Piau has: 239 entries.
Rio has: 715 entries.
Rondonia has: 239 entries.
Roraima has: 239 entries.
Santa Catarina has: 238 entries.
Sao Paulo has: 239 entries.
Sergipe has: 239 entries.
Tocantins has: 239 entries.

After dropping states with over 240 entries!

Acre has: 239 entries.
Alagoas has: 240 entries.
Amapa has: 239 entries.
Amazonas has: 239 entries.
Bahia has: 239 entries.
Ceara has: 239 entries.
Distrito Federal has: 239 entries.
Espirito Santo has: 238 entries.
Goias has: 238 entries.
Maranhao has: 239 entries.
Mato Grosso has: 0 entries.
Minas Gerais has: 237 entries.
Para has

In [280]:
#Given Test. This Cell Should Not Throw an Exception!
assert \
    len(amazon_fires['state'].unique()) == 20 and \
    list(amazon_fires['month'].unique()) == \
        ['January', 'February', 'March', 'April', 'May', 'June',
             'July', 'August','September', 'October', 'November',
             'December'] and \
    len(amazon_fires) == 4772, 'somethings wrong in problem 1.'

#### Part C: Medians and Means!
In this part of the problem, we'll calculate an 80% confidence interval for both the mean and median number of wildfires each state has during each month of the year. 

For the mean you should use the appropriate confidence interval with the correct distribution. Remember to check how many observations we have. Use the sample standard deviation. 

For the median, we'll have to bootstrap it because the median is not known to be normally distributed. You should bootstrap 1000 samples of the same length as the original sample for each month for each state. Calculate the median for each bootstrapped sample. Then take the middle 80% of the bootstrapped medians as your confidnce interval. This is called a bootstrapped percentile median. There are a few more complex and slightly more rigourous ways to estimate the median from bootstrapped samples, but this will serve for our purposes.

You're given a dictionary of dictionaries to store your confidence intervals for the medians and means in. 

Take a look at the dictionary structure below. 

The idea here is that for every month, for every state, you will fill in the `mean_CI` with a length two list that contains the low and high end of the confidence interval for the true mean number of fires for that state in that month. 

Similiarly, for every month, for every state, you will fill in the `median_CI` with a length two list that contains the low and high end of the confidence interval for the true median number of fires for that state in that month.

For example:

When you're done `months['January']['Acre']['mean_CI']` should be a list with the low and high bounds for the confidence interval of the true mean number of wildfires in the state of Acre in January. So `months['January']['Acre']['mean_CI'][0]` should be the low end of the CI for the mean, and `months['January']['Acre']['mean_CI'][1]` should be the high end of the CI for the mean.

`months['January']['Acre']['median_CI']` should hold the confidence interval for the true median number of wildfires in the state of Acre in January. So `months['January']['Acre']['median_CI'][0]` should be the low end of the CI for the median, and `months['January']['Acre']['median_CI'][1]` should be the high end of the CI for the median.

In [281]:
#GIVEN CODE DO NOT CHANGE THIS!!!
#YOU SHOULD BE WRITING CODE IN THE NEXT CELL(s) THAT FILLS IN THE 'months' DICTIONARY.

#If you're curious what copy and deep copy do and why we used them here see an explanation 
#here: https://thispointer.com/python-how-to-copy-a-dictionary-shallow-copy-vs-deep-copy/

from copy import deepcopy

mean_median_dict ={
    'mean_CI' : None,
    'median_CI': None
}

CI_median_num_fires = {
    'Acre': dict(mean_median_dict),
    'Alagoas':dict( mean_median_dict),
    'Amapa':dict( mean_median_dict),
    'Amazonas':dict( mean_median_dict),
    'Bahia':dict( mean_median_dict),
    'Ceara':dict( mean_median_dict),
    'Distrito Federal':dict( mean_median_dict),
    'Espirito Santo':dict( mean_median_dict),
    'Goias':dict( mean_median_dict),
    'Maranhao':dict( mean_median_dict),
    'Minas Gerais':dict( mean_median_dict),
    'Para':dict( mean_median_dict),
    'Pernambuco':dict( mean_median_dict),
    'Piau':dict( mean_median_dict),
    'Rondonia':dict( mean_median_dict),
    'Roraima':dict( mean_median_dict),
    'Santa Catarina':dict( mean_median_dict),
    'Sao Paulo':dict( mean_median_dict),
    'Sergipe':dict( mean_median_dict),
    'Tocantins':dict( mean_median_dict)  
}

months = {
    'January': deepcopy(CI_median_num_fires),
    'February': deepcopy(CI_median_num_fires),
    'March': deepcopy(CI_median_num_fires), 
    'April': deepcopy(CI_median_num_fires), 
    'May': deepcopy(CI_median_num_fires),
    'June': deepcopy(CI_median_num_fires),
    'July': deepcopy(CI_median_num_fires),
    'August': deepcopy(CI_median_num_fires), 
    'September': deepcopy(CI_median_num_fires), 
    'October': deepcopy(CI_median_num_fires),
    'November': deepcopy(CI_median_num_fires),
    'December': deepcopy(CI_median_num_fires)
}

In [282]:
#Your code here:
#Your code should modify the 'months' dictionary provided to add the CIs.

def calcMeanCI(alpha, data):
    data = data.to_numpy()
    return stats.t.interval(alpha, len(data)-1, loc=np.mean(data), scale=stats.sem(data))
def calcMedianCIBoot(data, numboots):
    data = data.to_numpy()
    boots = np.array([np.median(np.random.choice(data, replace=True, size=len(data))) for ii in range(numboots)])
    lower, upper = np.percentile(boots, 20), np.percentile(boots, 80)
    return np.array([lower, upper])
    
uniqueStates = amazon_fires['state'].unique()
uniqueMonths = amazon_fires['month'].unique()

for month in uniqueMonths:
    for state in uniqueStates:
        data = amazon_fires[(amazon_fires['state'] == state) & (amazon_fires['month'] == month)].index
        ciMean = calcMeanCI(0.8, (amazon_fires['number'].loc[data]))
        ciMedian = calcMedianCIBoot((amazon_fires['number'].loc[data]), 1000)

        months[month][state]['mean_CI'] = []
        months[month][state]['mean_CI'].append(ciMean[0])
        months[month][state]['mean_CI'].append(ciMean[1])
        
        months[month][state]['median_CI'] = []
        months[month][state]['median_CI'].append(ciMedian[0])
        months[month][state]['median_CI'].append(ciMedian[1])
        

In [283]:
#DONT CHANGE THIS. WE USE IT TO MAKE THE OUTPUT LEGIBLE FOR GRADING
import pprint
pp = pprint.PrettyPrinter(indent=1)
pp.pprint(months)

{'April': {'Acre': {'mean_CI': [0.7631702958582414, 3.336829704141758],
                    'median_CI': [0.0, 1.0]},
           'Alagoas': {'mean_CI': [7.979189204540319, 16.52081079545968],
                       'median_CI': [5.5, 13.0]},
           'Amapa': {'mean_CI': [0.3412383897376437, 0.9587616102623564],
                     'median_CI': [0.0, 0.0]},
           'Amazonas': {'mean_CI': [6.9737040719879255, 12.226295928012075],
                        'median_CI': [4.5, 11.0]},
           'Bahia': {'mean_CI': [93.8962665362685, 157.1037334637315],
                     'median_CI': [82.5, 109.0]},
           'Ceara': {'mean_CI': [2.695004965166694, 4.904995034833306],
                     'median_CI': [2.0, 4.0]},
           'Distrito Federal': {'mean_CI': [0.6266818228169977,
                                            1.8733181771830023],
                                'median_CI': [0.0, 1.0]},
           'Espirito Santo': {'mean_CI': [7.3942600245658525,
                    

In [284]:
#Given Test for the mean confidence intervals

rounded_mean_CI = [round(x, 2) for x in months['April']['Acre']['mean_CI']]
assert rounded_mean_CI == [0.76, 3.34], 'somethings wrong in the mean'

In [285]:
#Given test for the median confidence intervals. 
#Your code is probably correct if it passes this test, but since bootstrapping the medain is a stochastic process
#you may have this test fail. If it fails, run it a few times. 
#If it continues to fail, your code is probably incorrect.

low_median_CI = months['April']['Acre']['median_CI'][0]
high_median_CI = months['April']['Acre']['median_CI'][1]
assert -1 <= low_median_CI <= 1 and 0 <= high_median_CI <= 3, 'somethings wrong in the median'

#### Part E: Where Do The Firefighters Go?
Now, we'll determine which state the Brazilian government should assign it's fire fighters to. For each month of the year, you should perform the folllowing selection process:
1. Find the state with the highest CI for the median for this month (it's easiest and ok to just use the upper bound here). 
2. Find any states that have a median CI that overlaps with the highest CI foud in step 1. If no states overlap with the highest CI found in step 1, then use that stat. 
3. If overlapping confidence intervals are found on the median, we'll use the CI for the mean to break ties.
4. Out of the states with overlapping CIs for median (every state in part 3), find the state with the highest mean CI. 
5. Determine if any of the states from part 3 have a mean CI that overlaps with the state found in step 4. 
6. If no state overlap with the state found in part 4, then just use that state. If other states have overlapping mean CIs too, then we'll split up the firefighters and assign some of them to every state that has both an overlapping median and mean CI with the state that has the highest median CI.

Once you've used the selection process above, use a markdown table to display a list of each state that recieves  some of the firefighters for each month.

In [286]:
# Your code here.

uniqueStates = amazon_fires['state'].unique()
uniqueMonths = amazon_fires['month'].unique()

highestStates = []
for month in uniqueMonths:
    highestCIUpper = 0
    highestCILower = 0
    highestState = ''
    
    #find the highest state for this month
    for state in uniqueStates:
        if months[month][state]['median_CI'][1] > highestCIUpper:
            highestState = state
            highestCILower = months[month][state]['median_CI'][0]
            highestCIUpper = months[month][state]['median_CI'][1]
            
    #print(month+": " + highestState + " with [" + str(highestCILower) + "][" + str(highestCIUpper) + "]")
    
    #find if that state overlaps in median CI with any other states
    overlapping = []
    for state in uniqueStates:
        if state == highestState:
            continue
        if months[month][state]['median_CI'][1] > highestCILower:
            #A state overlaps this states CI
            overlapping.append(state)
            
    #If there are no overlapping states on the median CI        
    if len(overlapping) == 0:
        highestStates.append(state)
        continue
        
    overlapping.append(highestState)
    #Find state with highest mean CI
    highestCIUpper = 0
    highestCILower = 0
    highestState = ''

    for state in overlapping:
        if months[month][state]['mean_CI'][1] > highestCIUpper:
            highestState = state
            highestCILower = months[month][state]['mean_CI'][0]
            highestCIUpper = months[month][state]['mean_CI'][1]
    
    #check to see if any states that overlapped on the median CI overlap on the mean CI after the highest one is chosen
    meanOverlapping = []
    for state in overlapping:
        if state == highestState:
            continue
        if months[month][state]['mean_CI'][1] > highestCILower:
            #A state overlaps this states CI
            meanOverlapping.append(state)    
    
    #If no states overlapped then we will stick with this state
    if len(meanOverlapping) == 0:
        states = meanOverlapping + overlapping
        highestStates.append(states)
        continue
    highestStates.append(highestState)

for i in range(0, len(uniqueMonths)):
    tabs = '\t'
    if len(uniqueMonths[i]) < 7:
        tabs = '\t\t'
    print(uniqueMonths[i] + ":" +tabs+ "",highestStates[i])
        

January:	 Para
February:	 ['Bahia', 'Roraima']
March:		 Tocantins
April:		 Bahia
May:		 ['Bahia', 'Sao Paulo', 'Tocantins']
June:		 ['Maranhao', 'Tocantins']
July:		 Maranhao
August:		 Tocantins
September:	 Tocantins
October:	 Para
November:	 Tocantins
December:	 ['Maranhao', 'Para']


| Month | States   |
|------|------|
|   January  | Para |
|   February  | Bahia, Roraima |
|   March  | Tocantins |
|   April  | Bahia |
|   May  | Bahia, Sao Paula, Tocantins |
|   June  | Maranhao, Tocantins |
|   July  | Maranhao |
|   August  | Tocantins |
|   September  | Tocantins |
|   October  | Para |
|   November  | Tocantins |
|   December  | Maranhao, Para |

In [6]:
import math
print(stats.norm.interval(0.95, loc=743, scale=(5/math.sqrt(16))))

(740.5500450193249, 745.4499549806751)
